In [22]:
 %matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
train_df = pd.read_csv("data/2019loans.csv")
test_df = pd.read_csv("data/2020Q1loans.csv")

In [15]:
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [13]:
for col in train_df.columns:
    print(col)

Unnamed: 0
index
loan_amnt
int_rate
installment
home_ownership
annual_inc
verification_status
loan_status
pymnt_plan
dti
delinq_2yrs
inq_last_6mths
open_acc
pub_rec
revol_bal
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_amnt
collections_12_mths_ex_med
policy_code
application_type
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
chargeoff_within_12_mths
delinq_amnt
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
mths_since_recent_inq
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
nu

My prediction for this dataset is that Random Forest will be best, as the Random Forest Model tends to be more accurate overall and is more suited for these datatypes.

In [34]:
y_train = train_df['home_ownership']
X_train = train_df.drop(columns = ['home_ownership'])
X_train = pd.get_dummies(X_train)
X_train.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,57107,57107,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,...,1,1,0,1,1,0,1,0,1,0
1,141451,141451,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,...,1,1,0,1,1,0,1,0,1,0
2,321143,321143,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,...,1,1,0,1,1,0,1,0,1,0
3,11778,11778,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,...,1,1,0,1,1,0,1,0,1,0
4,169382,169382,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,...,1,1,0,1,1,0,1,0,1,0


In [35]:
y_test = test_df['home_ownership']
X_test = test_df.drop(columns = ['home_ownership'])
X_test = pd.get_dummies(X_test)

In [36]:
for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0

In [40]:
# train the logisitic regression model on unscaled data

regression = LogisticRegression().fit(X_train, y_train)
regression.score(X_test, y_test)
print (f' Unscaled Regression Score : {regression.score(X_test, y_test)}')

 Unscaled Regression Score : 0.7662696724797958


C:\Users\elsmi\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [41]:
# train random forest on uncscaled data
clf = RandomForestClassifier(random_state=42, n_estimators=100).fit(X_train, y_train)

In [45]:
print (f' Unscaled Random Forest Score : {clf.score(X_test, y_test)}')

 Unscaled Random Forest Score : 0.7722245852828583


Following the initial training and testing of the models, I'm surprised to see that the scores are very similar. I believe scaling will improve the Logistic Regression model as scaling is not generally used in Random Forest Classifiers.

In [47]:
# Scale the data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [52]:
scaled_regression = LogisticRegression().fit(X_train_scaled, y_train)
scaled_regression.score(X_test_scaled, y_test)
print (f' Scaled Regression Score : {scaled_regression.score(X_test_scaled, y_test)}')

 Scaled Regression Score : 0.7705231816248405


C:\Users\elsmi\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [53]:
scaled_clf = RandomForestClassifier(random_state=42, n_estimators=100).fit(X_train_scaled, y_train)

In [55]:
print (f' Scaled Random Forest Score : {scaled_clf.score(X_test_scaled, y_test)}')

 Scaled Random Forest Score : 0.7730752871118672


In [67]:
print (f' Unscaled Regression Score : {regression.score(X_test, y_test)}\n'
       f' Scaled Regression Score : {scaled_regression.score(X_test_scaled, y_test)}')
print (f' Unscaled Random Forest Score : {clf.score(X_test, y_test)}\n'
       f' Scaled Random Forest Score : {scaled_clf.score(X_test_scaled, y_test)}')
       


 Unscaled Regression Score : 0.7662696724797958
 Scaled Regression Score : 0.7705231816248405
 Unscaled Random Forest Score : 0.7722245852828583
 Scaled Random Forest Score : 0.7730752871118672


The scores for both Linear Regression and Random Forest remained consistent through the scaling process. The scores were also very similar between the two models. The Linear Regression improved slightly with the scaling, whereas the Random Forest stayed almost exactly the same. The Random Forest behaved as expected through the scaling, however I did anticipate more improvement to the Logisitic Regression model.

What does this actually say for which is the better model? I am unsure as to which is better as the scores are so close, perhaps more testing with different target features may illuminate which is better for this dataset.